# script per generare id impresa univoco e pseudonimizzato

i dati ..

 



## setup



In [1]:
# Setup
import sys
import os
from pathlib import Path
import datetime
import pandas as pd
pd.options.display.max_columns = None

In [2]:
current_path = Path(os.getcwd())
data_subdir = "dati_imprese"
data_path = current_path.parent / data_subdir
data_dir = str(data_path)
listafile = os.listdir(data_path)
listafile = list(filter(lambda x: 'imprese_' in x, listafile)) #filtra solo i file che contengono IMPRESE
print(listafile)
assert len(listafile) == 2, "Ci sono più di due file imprese_*.csv"



['imprese_anagrafica.csv', 'imprese_codici.csv']


## genero la tabella 1: identificazione univoca delle imprese con CF


In [3]:
cols_date = ['data_isc_ri', 'data_isc_rd', 'data_isc_aa', 'data_apert_ul', 'data_ini_at']
cols_to_use =   ['cf'] + cols_date
dtypes = [ 'str','str','str','str','str'  ,'str' ,'str' ]

cols_date
 

['data_isc_ri', 'data_isc_rd', 'data_isc_aa', 'data_apert_ul', 'data_ini_at']

In [4]:
filename = data_dir +'\\imprese_anagrafica.csv'
df = pd.read_csv (  filename, 
                    sep = '|', 
                    usecols = cols_to_use ,
                    dtype=str,
                    keep_default_na=False              
)
df.replace('', 'NaT', inplace=True)
df.replace('#ERRORE', 'NaT', inplace=True)
df


,cf,data_isc_ri,data_isc_rd,data_isc_aa,data_ini_at,data_apert_ul
0,00000470310,4996-02-19 00:00:00,4975-01-14 00:00:00,NaT,NaT,NaT
1,00002070324,4996-02-19 00:00:00,4969-01-30 00:00:00,NaT,4969-01-30 00:00:00,NaT
2,00002070324,4996-02-19 00:00:00,4969-01-30 00:00:00,NaT,NaT,5007-08-01 00:00:00
3,00002070324,4996-02-19 00:00:00,4969-01-30 00:00:00,NaT,NaT,5015-10-15 00:00:00
4,00002130938,4996-02-19 00:00:00,4973-10-31 00:00:00,NaT,NaT,NaT
...,...,...,...,...,...,...
127169,ZZZNTN78H14L195O,5009-03-10 00:00:00,5009-03-10 00:00:00,5009-03-10 00:00:00,5009-03-10 00:00:00,NaT
127170,ZZZRRT70L67L424I,5008-02-21 00:00:00,5008-02-19 00:00:00,5008-02-01 00:00:00,5008-02-01 00:00:00,NaT
127171,ZZZRRT74D06Z700T,4998-01-27 00:00:00,4998-01-15 00:00:00,NaT,4998-01-01 00:00:00,NaT
127172,ZZZSVN46E60G381I,4996-12-02 00:00:00,4996-12-02 00:00:00,NaT,4991-03-09 00:00:00,NaT


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127174 entries, 0 to 127173
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   cf             127174 non-null  object
 1   data_isc_ri    127174 non-null  object
 2   data_isc_rd    127174 non-null  object
 3   data_isc_aa    127174 non-null  object
 4   data_ini_at    127174 non-null  object
 5   data_apert_ul  127174 non-null  object
dtypes: object(6)
memory usage: 5.8+ MB


In [6]:
#anni spostati di +3000
#da rifare con fillna per i valori nulli e con list comprehension per velocità
lista_date_impossibili = []

for index, row in df.iterrows():
    for col in cols_date:
        #print(index, row, col)
        
        item = df.loc[index, col]
        #print(item)
        if isinstance(item, str):
            temp = item[0:2]  
            #print(temp) 
        else:
            break
        
        if temp == 'Na':
            pass#print(col,temp,row[col] )
        elif temp == '48':
            df.loc[index, col] = str('18') + str(df.loc[index, col][2:10])
        elif temp == '49':
            df.loc[index, col] = str('19') + str(df.loc[index, col][2:10])
        elif temp == '50':
            df.loc[index, col] = str('20') + str(df.loc[index, col][2:10])
        else:
            print('Valori non coerenti ', item)
            lista_date_impossibili.append(item)
            df.loc[index, col] = 'NaT'

             
        


Valori non coerenti  3013-02-01 00:00:00
Valori non coerenti  5105-02-28 00:00:00
Valori non coerenti  4595-01-01 00:00:00
Valori non coerenti  5102-02-22 00:00:00
Valori non coerenti  3202-02-02 00:00:00
Valori non coerenti  3215-01-31 00:00:00
Valori non coerenti  3195-08-01 00:00:00
Valori non coerenti  6007-02-09 00:00:00


In [7]:
for col in cols_date:
    df[col] = pd.to_datetime(df[col])

df

,cf,data_isc_ri,data_isc_rd,data_isc_aa,data_ini_at,data_apert_ul
0,00000470310,1996-02-19,1975-01-14,NaT,NaT,NaT
1,00002070324,1996-02-19,1969-01-30,NaT,1969-01-30,NaT
2,00002070324,1996-02-19,1969-01-30,NaT,NaT,2007-08-01
3,00002070324,1996-02-19,1969-01-30,NaT,NaT,2015-10-15
4,00002130938,1996-02-19,1973-10-31,NaT,NaT,NaT
...,...,...,...,...,...,...
127169,ZZZNTN78H14L195O,2009-03-10,2009-03-10,2009-03-10,2009-03-10,NaT
127170,ZZZRRT70L67L424I,2008-02-21,2008-02-19,2008-02-01,2008-02-01,NaT
127171,ZZZRRT74D06Z700T,1998-01-27,1998-01-15,NaT,1998-01-01,NaT
127172,ZZZSVN46E60G381I,1996-12-02,1996-12-02,NaT,1991-03-09,NaT


In [8]:
lista_date_impossibili

['3013-02-01 00:00:00',
 '5105-02-28 00:00:00',
 '4595-01-01 00:00:00',
 '5102-02-22 00:00:00',
 '3202-02-02 00:00:00',
 '3215-01-31 00:00:00',
 '3195-08-01 00:00:00',
 '6007-02-09 00:00:00']

In [9]:
df['min_date'] = df[cols_date].min(axis=1)
df

,cf,data_isc_ri,data_isc_rd,data_isc_aa,data_ini_at,data_apert_ul,min_date
0,00000470310,1996-02-19,1975-01-14,NaT,NaT,NaT,1975-01-14
1,00002070324,1996-02-19,1969-01-30,NaT,1969-01-30,NaT,1969-01-30
2,00002070324,1996-02-19,1969-01-30,NaT,NaT,2007-08-01,1969-01-30
3,00002070324,1996-02-19,1969-01-30,NaT,NaT,2015-10-15,1969-01-30
4,00002130938,1996-02-19,1973-10-31,NaT,NaT,NaT,1973-10-31
...,...,...,...,...,...,...,...
127169,ZZZNTN78H14L195O,2009-03-10,2009-03-10,2009-03-10,2009-03-10,NaT,2009-03-10
127170,ZZZRRT70L67L424I,2008-02-21,2008-02-19,2008-02-01,2008-02-01,NaT,2008-02-01
127171,ZZZRRT74D06Z700T,1998-01-27,1998-01-15,NaT,1998-01-01,NaT,1998-01-01
127172,ZZZSVN46E60G381I,1996-12-02,1996-12-02,NaT,1991-03-09,NaT,1991-03-09


In [10]:
mindate = min(df.min_date)
mindate


# calcola una data iniziale
# ordina in base alla data
#poi crea l'elenco di cf unique ordinato in base alla data

Timestamp('1800-01-01 00:00:00')

In [11]:
maxdate = max(df.min_date)
maxdate

Timestamp('2021-04-12 00:00:00')

In [12]:
today = datetime.date(2021,5,1) 
print(type(today))
df['today'] = today 

df['anni'] = 0.0
df['anni'] =  (df.today - df.min_date.dt.date)
print(df.anni)
 

<class 'datetime.date'>
0        16909 days
1        19084 days
2        19084 days
3        19084 days
4        17349 days
            ...    
127169    4435 days
127170    4838 days
127171    8521 days
127172   11011 days
127173    7928 days
Name: anni, Length: 127174, dtype: timedelta64[ns]


In [13]:
df.sort_values(by = 'anni', ascending=False, inplace = True)
df

,cf,data_isc_ri,data_isc_rd,data_isc_aa,data_ini_at,data_apert_ul,min_date,today,anni
55165,02532430309,2008-07-02,2008-07-01,NaT,1800-01-01,NaT,1800-01-01,2021-05-01,80839 days
4596,00267590321,NaT,1992-02-17,NaT,1829-11-11,NaT,1829-11-11,2021-05-01,69933 days
67465,80001930306,NaT,1927-03-28,NaT,1881-01-01,NaT,1881-01-01,2021-05-01,51254 days
39127,01738760303,1996-02-19,1991-09-11,NaT,1881-09-02,NaT,1881-09-02,2021-05-01,51010 days
4072,00249360306,1996-02-19,1925-10-10,NaT,1887-02-01,NaT,1887-02-01,2021-05-01,49032 days
...,...,...,...,...,...,...,...,...,...
29441,01359640321,2021-04-13,2021-04-12,NaT,NaT,NaT,2021-04-12,2021-05-01,19 days
65910,03031730306,2021-04-13,2021-04-12,NaT,NaT,NaT,2021-04-12,2021-05-01,19 days
29414,01358950325,2021-04-13,2021-04-12,NaT,NaT,NaT,2021-04-12,2021-05-01,19 days
29450,01359920327,2021-04-12,2021-04-12,NaT,NaT,NaT,2021-04-12,2021-05-01,19 days


In [14]:
#semplice elenco di cf con duplicati
cf_list_all = df['cf'].tolist()
print(len(cf_list_all))

#elenco cf senza duplicati
cf_list_unique = df['cf'].unique().tolist()
print(len(cf_list_unique))



127174
103072


In [15]:
print(df.shape[0])
df.nunique()

127174


cf               103072
data_isc_ri        6438
data_isc_rd       13797
data_isc_aa        9801
data_ini_at       15365
data_apert_ul      7698
min_date          15881
today                 1
anni              15881
dtype: int64

In [16]:
df.reset_index(inplace=True)
df.drop(columns = ['index'], inplace=True)
df.reset_index(inplace=True)
df.rename(columns={"index":"id_impresa"}, inplace=True)
df.id_impresa += 1

df

,id_impresa,cf,data_isc_ri,data_isc_rd,data_isc_aa,data_ini_at,data_apert_ul,min_date,today,anni
0,1,02532430309,2008-07-02,2008-07-01,NaT,1800-01-01,NaT,1800-01-01,2021-05-01,80839 days
1,2,00267590321,NaT,1992-02-17,NaT,1829-11-11,NaT,1829-11-11,2021-05-01,69933 days
2,3,80001930306,NaT,1927-03-28,NaT,1881-01-01,NaT,1881-01-01,2021-05-01,51254 days
3,4,01738760303,1996-02-19,1991-09-11,NaT,1881-09-02,NaT,1881-09-02,2021-05-01,51010 days
4,5,00249360306,1996-02-19,1925-10-10,NaT,1887-02-01,NaT,1887-02-01,2021-05-01,49032 days
...,...,...,...,...,...,...,...,...,...,...
127169,127170,01359640321,2021-04-13,2021-04-12,NaT,NaT,NaT,2021-04-12,2021-05-01,19 days
127170,127171,03031730306,2021-04-13,2021-04-12,NaT,NaT,NaT,2021-04-12,2021-05-01,19 days
127171,127172,01358950325,2021-04-13,2021-04-12,NaT,NaT,NaT,2021-04-12,2021-05-01,19 days
127172,127173,01359920327,2021-04-12,2021-04-12,NaT,NaT,NaT,2021-04-12,2021-05-01,19 days


In [17]:
df.drop(columns = cols_date, inplace=True)
df.drop(columns = ['min_date', 'today'], inplace=True)
df

,id_impresa,cf,anni
0,1,02532430309,80839 days
1,2,00267590321,69933 days
2,3,80001930306,51254 days
3,4,01738760303,51010 days
4,5,00249360306,49032 days
...,...,...,...
127169,127170,01359640321,19 days
127170,127171,03031730306,19 days
127171,127172,01358950325,19 days
127172,127173,01359920327,19 days


In [18]:
file_risultati  = data_dir + '\\' + 'insiel_pseudon_cf.csv'
df.to_csv(file_risultati, 
                    sep ='|',   
                    encoding='utf-8-sig', 
                    index=False)

In [19]:
df.drop_duplicates(subset=['cf'], inplace=True)
df.drop(columns = ['id_impresa'], inplace=True)
df

,cf,anni
0,02532430309,80839 days
1,00267590321,69933 days
2,80001930306,51254 days
3,01738760303,51010 days
4,00249360306,49032 days
...,...,...
127169,01359640321,19 days
127170,03031730306,19 days
127171,01358950325,19 days
127172,01359920327,19 days


In [20]:
df

,cf,anni
0,02532430309,80839 days
1,00267590321,69933 days
2,80001930306,51254 days
3,01738760303,51010 days
4,00249360306,49032 days
...,...,...
127169,01359640321,19 days
127170,03031730306,19 days
127171,01358950325,19 days
127172,01359920327,19 days


In [21]:
df.reset_index(inplace=True)
df.rename(columns={"index":"id_impresa"}, inplace=True)
df.id_impresa += 1
file_risultati  = data_dir + '\\' + 'insiel_pseudon_cf_unique.csv'
df.to_csv(file_risultati, 
                    sep ='|',   
                    encoding='utf-8-sig', 
                    index=False)

In [22]:
df

,id_impresa,cf,anni
0,1,02532430309,80839 days
1,2,00267590321,69933 days
2,3,80001930306,51254 days
3,4,01738760303,51010 days
4,5,00249360306,49032 days
...,...,...,...
103067,127170,01359640321,19 days
103068,127171,03031730306,19 days
103069,127172,01358950325,19 days
103070,127173,01359920327,19 days
